In [1]:
import librosa
from librosa import feature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
import graphviz 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
def extract_features(file):

    # check if normal or abnormal
    if 'abnormal' in file:
        operation=1
    else:
        operation=0
    # loading the file, getting y and sr (sample rate)

    y, sr = librosa.load(file)

    # Getting S and phase

    S, phase = librosa.magphase(librosa.stft(y=y))

    # Features for the DataFrame
# chroma features have pitch which is probably not usefull
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))

    chroma_cqt = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr))

    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr))
# to do
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_min = np.min(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_max = np.max(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
    melspectrogram_sum = librosa.feature.melspectrogram(y=y, sr=sr, S=S).sum()
    melspectrogram_corr= np.mean(np.corrcoef(librosa.feature.melspectrogram(y=y, sr=sr, S=S)))
    melspectrogram_std= np.std(librosa.feature.melspectrogram(y=y, sr=sr, S=S))
# to check This feature is one of the most important method to extract a feature of an audio signal and is 
#used majorly whenever working on audio signals. The mel frequency cepstral coefficients (MFCCs) of a signal 
#are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral 
#envelope.
    
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr))
#rms when plotted similarities with melspectorgram
#normal rms mean higher than abnormal?
    rms = np.mean(librosa.feature.rms(y=y, S=S))
#spectral centroid computes weighted mean of the frequencies in the sound
#plot is line on melspectogram
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, S=S))
#Bandwidth(blue zone) is the difference between the upper and lower frequencies in a continuous band of frequencies
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, S=S))
#needs further investigation, but makes continous data discrete in bins
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, S=S))
#Spectral flatness (or tonality coefficient) is a measure to quantify how much noise-like a sound is, as opposed to
#being tone-like 1. A high spectral flatness (closer to 1.0) indicates the spectrum is similar to white noise. 
#It is often converted to decibel.
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y, S=S))
#The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at 
#least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and 
#he bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent 
#to a value close to 1 (or 0).
#rolloff with rolloff coefficient 0.01 seems to be the same for (ab)normal
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, S=S))
#Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram
    poly_features = np.mean(librosa.feature.poly_features(y=y, sr=sr, S=S))
#probably chroma
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr))
#normal has higher zero crossing rate?
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
#divide in frequency bands could give more information bout difference normal and abnormal (BPM)
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr,
                       #    hop_length=hop_length)[0]
    # putting them into a dataframe
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    return pd.DataFrame({ 'melspectrogram' : [melspectrogram],'melspectrogram_min':[melspectrogram_min],
                             'melspectrogram_max':[melspectrogram_max], 'melspectrogram_sum':[melspectrogram_sum],
                             'melspectrogram_corr':[melspectrogram_corr] ,'melspectrogram_std':[melspectrogram_std] ,
                             'mfcc' : [mfcc], 'rms' : [rms],
                            'spectral_centroid' : [spectral_centroid], 'spectral_bandwidth' : [spectral_bandwidth],
                            'spectral_contrast' : [spectral_contrast], 'spectral_flatness ' : [spectral_flatness],
                            'spectral_rolloff' : [spectral_rolloff], 
                            'zero_crossing_rate' : [zero_crossing_rate],"mean harm": np.mean(y_harmonic),
                            "mean perc": [np.mean(y_percussive)],"max harm":[np.max(y_harmonic)],"max perc": [np.max(y_percussive)],
                             "min harm":[np.min(y_harmonic)], "min perc":[np.min(y_percussive)], 'normal(0)/abnormal(1)':[operation]})

In [4]:
import os
import sys
import time
def create_csv(machine):
    # start measuring excecution time
    start_time = time.time()
    #create empty list that will be filled with pathnames
    #find paths normal wav files
    list_normal_6 = []
    num=[0,2,4,6]
    for i in num:
        directory_normal_6 = f"/home/regis/Desktop/Sound Project/files/{machine}/6_dB_{machine}/{machine}/id_0{i}/normal/"
        for filename in os.listdir(directory_normal_6):
            file = f"{directory_normal_6}{filename}"
            list_normal_6.append(file)
    list_normal_6.sort()
    #find paths abnormal wav files
    list_abnormal_6 = []
    num=[0,2,4,6]
    for i in num:
        directory_abnormal_6 = f"/home/regis/Desktop/Sound Project/files/{machine}/6_dB_{machine}/{machine}/id_0{i}/abnormal/"
        for filename in os.listdir(directory_abnormal_6):
            file = f"{directory_abnormal_6}{filename}"
            list_abnormal_6.append(file)
    list_abnormal_6.sort()
    #add normal to df
    for wav_file in list_normal_6:
        df = extract_features(wav_file)
        if wav_file == list_normal_6[0]:
            df.to_csv(f'Librosa_features_{machine}.csv')
        else:
            df.to_csv(f'Librosa_features_{machine}.csv', mode='a', header=False)
    #add abnormal to df
    for wav_file in list_abnormal_6:
        df = extract_features(wav_file)
        df.to_csv(f'Librosa_features_{machine}.csv', mode='a', header=False) 
    print("--- %s seconds ---" % (time.time() - start_time))

In [5]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    

    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = pd.concat(data_frames)
    
    
    return df_merged

In [6]:
merge_pd('valve')

,melspectrogram,melspectrogram_min,melspectrogram_max,melspectrogram_sum,melspectrogram_corr,melspectrogram_std,mfcc,rms,spectral_centroid,spectral_bandwidth,...,spectral_flatness,spectral_rolloff,zero_crossing_rate,mean harm,mean perc,max harm,max perc,min harm,min perc,normal(0)/abnormal(1)
0,0.006300,6.218734e-10,0.090277,347.580261,0.523183,0.010437,-15.342528,0.002967,1399.372425,1635.526256,...,0.000852,2847.104130,0.060725,-3.781356e-07,4.012865e-07,0.006918,0.070441,-0.006861,-0.060939,0
1,0.007236,7.803309e-10,0.087623,399.178070,0.461858,0.009354,-14.877153,0.002920,1765.622412,1767.165689,...,0.001012,3777.878084,0.094482,-1.201143e-07,-3.884470e-07,0.006952,0.070928,-0.006763,-0.060128,0
2,0.006186,6.056689e-10,0.092516,341.284851,0.494020,0.008715,-16.809891,0.002680,1716.821280,1784.159467,...,0.000950,3394.926803,0.094802,-3.535212e-07,-4.008645e-07,0.007342,0.070832,-0.007282,-0.060499,0
3,0.006994,6.636679e-10,0.097133,385.858826,0.489502,0.009895,-15.353941,0.002953,1466.548926,1585.731245,...,0.000681,3035.257362,0.071383,-1.655043e-09,-1.397088e-07,0.006073,0.073961,-0.005455,-0.060447,0
4,0.005965,5.532336e-10,0.119931,329.072296,0.520290,0.010416,-15.100811,0.002925,1267.393978,1510.835471,...,0.000508,2481.789092,0.053385,-7.166033e-07,1.267004e-07,0.006912,0.070822,-0.006740,-0.063913,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165,0.019487,2.489299e-09,0.635858,1075.042847,0.177583,0.031520,-10.045176,0.008794,1641.324695,1865.835460,...,0.000368,3646.580502,0.081652,-5.208518e-07,-1.669572e-06,0.034763,0.050737,-0.029444,-0.034433,1
4166,0.023151,2.788397e-09,0.368241,1277.182617,0.142403,0.034830,-8.458351,0.009681,1290.344990,1553.914409,...,0.000242,2850.176733,0.056757,1.597554e-07,-2.796939e-06,0.023207,0.040186,-0.021190,-0.036905,1
4167,0.022438,2.718687e-09,0.332574,1237.873291,0.157012,0.035486,-8.748610,0.009778,1185.536634,1482.912767,...,0.000213,2594.775957,0.050464,1.254795e-06,2.009042e-06,0.024952,0.032241,-0.025619,-0.029852,1
4168,0.019146,2.529336e-09,0.379696,1056.230225,0.189193,0.031191,-11.065845,0.008707,1525.979681,1819.205808,...,0.000313,3295.604279,0.073881,1.195620e-07,-3.352579e-08,0.031278,0.034014,-0.028128,-0.032567,1
